# Основы Pandas

In [2]:
import pandas as pd

В течение данного занятия будем сравнивать удобство и скорость выполения ряда операций с помощью стандартных методов и библиотеки 
` pandas`.<br>


Для начала работы нам понадобится создать хранилище с данными. Пусть будет 3 физические величины. Создадим три массива, где будут храниться значения, которые в теории могут принимать эти физические величины.  

Значения тока должны лежать в пределах от 20 до 30 с шагом в 1, значения для магнитного поля в пределах от 40 до 60 с шагом в 2.  

In [4]:
current_all = []
field_all = []

current_all = list(range(20, 31, 1))
field_all  = list(range(40,61,2))

Теперь представим, что мы провели реальный физический эксперимент, в котором меняли поле и ток в заданных диапазонах (значения могут повторяться) и измеряли значение напряжение. 

Напишите функцию, которая рандомно выбирает значение тока и магнитного поля их списков `current_all` и `field_all` и генерирует  рандомное значение напряжения в пределах от 0 В до 100 В. 

На вход подаются два массива значений (current_all, field_all). На выходе список из трех элементов: ток, магнитное поле, напряжение. 

In [5]:
from random import choice, uniform

def gen_values(current=current_all , field = field_all) -> list:
    row = []
    row.append(choice(current))
    row.append(choice(field))
    row.append(uniform(0, 100))
    return row
    
def gen_values_1(current=current_all , field = field_all) -> list:
    return [choice(current), choice(field), uniform(0, 100)]

Теперь можем приступить к формированию баз данных. Каждый способ будем применять поочерёдно, измеряя время его выполнения. `time.perf_counter()` поможет в этом. 

Для увеличения точности эксперимента создайте словарь, содержащий три физические велечины, каждой из которых соотвествует  список из M = 1000 значений. Получите время выполнения данного примера. 


In [6]:
from time import perf_counter

# Число измерений
N = 1000  

In [7]:
start = perf_counter()


dict_storage = {'H': [],
                'I': [],
                'U': []}

for _ in range(N):
    data = gen_values()
    [dict_storage[value].append(data[j]) for j, value in enumerate(dict_storage)]

for _ in range(N):
    data = gen_values()
    dict_storage['H'].append(data[0])
    dict_storage['I'].append(data[1])
    dict_storage['U'].append(data[2])

print(perf_counter() - start)

0.001934100000653416


In [9]:
start = perf_counter()

pandas_storage = pd.DataFrame(columns=['H', 'I', 'U'])
for _ in range(N):
    d = pd.DataFrame([gen_values(),], columns=['H', 'I', 'U'])
    pandas_storage = pd.concat([pandas_storage, d], ignore_index=True)

print(perf_counter() - start)

0.15341389999957755


C:\Users\armix\AppData\Local\Temp\ipykernel_19228\2179834015.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas_storage = pd.concat([pandas_storage, d], ignore_index=True)


Разница в 100 раз сразу даёт понять, что лучше использовать словарь, а уже после этого преобразовывать его в DataFrame

Потом можете попробовать придумать более быстрый алгоритм, а мы перейдём к работе с матрицами. Среди любых данных будет присутсвовать шум. Предположим, что у нас он постоянный, поэтому нас интересуют значения тока и поля, при которых напряжение было больше 3. Найдём нужные нам строки

In [10]:
pandas_storage.loc[pandas_storage['U'] > 5]

,H,I,U
0,22,40,65.740116
1,29,46,86.561450
2,21,58,38.989143
3,21,46,47.946905
4,27,48,57.149684
...,...,...,...
995,21,50,7.659317
996,22,50,77.293973
997,20,42,74.298449
998,27,52,37.348337


Получили меньшую по вертикальному размеру матрицу, но с неправильной нумирацией строк. Восстановим её методом `df.reset_index()`

In [11]:
pandas_storage.loc[pandas_storage['U'] > 5].reset_index()

,index,H,I,U
0,0,22,40,65.740116
1,1,29,46,86.561450
2,2,21,58,38.989143
3,3,21,46,47.946905
4,4,27,48,57.149684
...,...,...,...,...
952,995,21,50,7.659317
953,996,22,50,77.293973
954,997,20,42,74.298449
955,998,27,52,37.348337


Почти получили требуемые данные, удалим лишний столбец методом `df.drop(label, axis=0, inplace=False,...)`

In [12]:
pandas_storage_new = pandas_storage.loc[pandas_storage['U'] > 5].reset_index().drop('index', axis=1)
pandas_storage_new

,H,I,U
0,22,40,65.740116
1,29,46,86.561450
2,21,58,38.989143
3,21,46,47.946905
4,27,48,57.149684
...,...,...,...
952,21,50,7.659317
953,22,50,77.293973
954,20,42,74.298449
955,27,52,37.348337


Теперь найдём среднее значение напряжение при каждой комбинации полей и тока, например, для построения трёхмерного графика

In [239]:
pandas_storage_new.groupby(['H', 'I']).mean()

U
H  I            
20 40  51.513941
   42  42.064408
   44  64.465407
   46  39.592509
   48  67.626537
...          ...
30 52  50.840503
   54  44.626392
   56  66.239735
   58  56.958432
   60  48.782562

[121 rows x 1 columns]

И переведём данные обратно в словарь

In [240]:
pandas_storage_new.groupby(['H', 'I']).mean().to_dict()

{'U': {(20, 40): 51.51394115247767,
  (20, 42): 42.06440761878909,
  (20, 44): 64.46540748625894,
  (20, 46): 39.59250899205013,
  (20, 48): 67.62653672724967,
  (20, 50): 72.01007537942387,
  (20, 52): 49.3053392808179,
  (20, 54): 64.70691025497841,
  (20, 56): 60.78663301965168,
  (20, 58): 24.095716475636134,
  (20, 60): 75.36746225844182,
  (21, 40): 51.310615495814275,
  (21, 42): 34.33074776109933,
  (21, 44): 63.868489627097425,
  (21, 46): 58.21225992866439,
  (21, 48): 58.1869599479831,
  (21, 50): 42.34155121697316,
  (21, 52): 55.96480364453748,
  (21, 54): 59.06309152790738,
  (21, 56): 44.53289409151393,
  (21, 58): 43.081236794355135,
  (21, 60): 79.38343488477263,
  (22, 40): 71.99837917881013,
  (22, 42): 27.9345698002787,
  (22, 44): 30.747402309816923,
  (22, 46): 50.722421744901574,
  (22, 48): 55.91571442125473,
  (22, 50): 50.670775226059725,
  (22, 52): 32.69825793296415,
  (22, 54): 57.056980589379926,
  (22, 56): 45.43545150724766,
  (22, 58): 63.55245096963799

Также можно отдельные столбцы и строки преобразовывать в списки (`Serial.to_list()`)

Для исследования зависимости значения физической величины от двух других параметров, можно создать матрицу, используя значения 
этих величин в качестве индексов. Используем уже созданный pandas_storage_new и его сгруппированную версию.  В качестве индексов будем испольовать значения поля, а для названий столбцов возьмем значения тока. 





In [13]:
print(pandas_storage['H'].unique())
print(pandas_storage['I'].unique())

[22 29 21 27 26 30 20 25 23 24 28]
[40 46 58 48 42 52 60 54 56 50 44]


Очевидно, что метод `.unique()` вернёт набор уникальных значений, причём в порядке следования в хранилище. Удобнее держать его отсортированным. Используем фунцкцию из библиотеки Numpy, так как все массивы являются массивами из неё. Подробнее об этом в следующей главе

In [12]:
import numpy as np

print(indexes := np.sort(pandas_storage['H'].unique()))
print(columns := np.sort(pandas_storage['I'].unique()))

massive = pd.DataFrame(index=indexes, columns=columns)
massive

[20 21 22 23 24 25 26 27 28 29 30]
[40 42 44 46 48 50 52 54 56 58 60]


,40,42,44,46,48,50,52,54,56,58,60
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Раз создали матрицу, пора заполнять её числами! У нас есть словарь, ключами которого являются пара значений (индекс, столбец), которым соотвествует искомое значение напряжения, остаётся правильно расположить его. Для заполнения используйте метод `df.loc[index, column] = ...` Решение может быть реализовано одной строкой)

In [14]:
for keys, value in pandas_storage.groupby(['H', 'I']).mean().to_dict()['U'].items():
    massive.loc[keys[0], keys[1]] = value
massive

,40,42,44,46,48,50,52,54,56,58,60
20,53.663382,29.378182,47.134578,50.089578,66.892839,46.233704,56.475624,45.309844,53.409478,53.620644,33.331867
21,51.516376,49.741414,47.636425,53.694068,56.408602,49.299699,59.407231,50.35724,40.333757,55.798314,73.917365
22,68.36277,52.272167,28.592442,48.0852,58.352807,53.810732,69.01896,40.13532,48.530152,65.480992,58.975729
23,45.336768,41.000036,39.32279,48.639041,39.461481,44.74343,42.004557,57.437397,54.80393,52.886569,49.677412
24,53.286972,44.174288,63.109703,50.448069,41.782014,48.738093,65.198941,63.224729,56.397928,34.882102,59.48841
25,42.682263,61.35281,54.637582,53.196233,51.974132,11.661466,60.777314,56.509511,64.316239,40.195179,51.997359
26,51.315163,18.656666,39.878358,49.241803,47.744327,62.088014,55.530248,37.991126,60.126926,51.074665,59.15523
27,59.549129,40.893573,60.548431,52.763451,32.882388,47.631575,28.969737,45.389286,84.730016,54.964369,25.371496
28,45.771681,62.188391,44.15957,50.113734,48.666355,68.787378,41.947803,73.742445,52.448886,44.531363,47.936378
29,66.677137,72.918335,43.558296,46.587981,36.774964,49.332451,48.625557,51.024784,61.312502,49.708084,41.322815


**Задача 2.3.** Из имеющегося массива данных вычислите среднее значение сопротивления при каждом из параметров, кроме нулевого тока. 

Создайте общий DataFrame для трех обрацов. Индексы в DataFrame должны соответствовать значениям температуры. 

In [244]:
Sample_1 = {'T': [300, 300, 300, 250, 250, 200, 200, 200, 150, 150, 150, 150, 100],
            'I': [0, 10, 20, 0, 10, 0, 10, 20, 0, 10, 20, 20, 0],
            'R': [1, 4, 7, 1, 6, 1, 8, 15, 1, 11, 21, 22, 1]}
Sample_2 = {'T': [300, 300, 300, 300, 250, 250, 200, 200, 200, 150, 150, 150, 100],
            'I': [0, 30, 40, 0, 40, 0, 30, 40, 0, 30, 40, 40, 0],
            'R': [0, 4, 8, 7, 0, 6, 0, 7, 14, 0, 10, 20, 0]}
Sample_3 = {'T': [300, 300, 300, 250, 250, 200, 200, 200, 150, 150, 150, 150, 100],
            'I': [0, 10, 20, 0, 10, 0, 10, 20, 0, 10, 20, 30, 0],
            'R': [1, 2, 3, 1, 4, 7, 1, 6, 1, 7, 12, 18, 1]}

Поступал вопрос про вычисление корреляции двух последовательностей. В `Pandas` есть такой метод: `df.corr()` принимает массив из *DF* и вычисляет действительное число зависимости содержимого от строки, к которой применён метод

In [ ]:
import pandas as pd
data = { 'Region': ['North', 'South', 'East', 'West'], 'Sales': [200, 150, 300, 250], 'Advertising': [50, 40, 70, 60] }
df = pd.DataFrame(data)
correlation = df['Sales'].corr(df['Advertising'])
float(correlation)

1.0